In [ ]:
!pip install face_recognition

In [ ]:
import face_recognition

In [ ]:
import os

In [ ]:
import cv2

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Step1. Register Image

In [ ]:
KNOWN_FACES_DIR = "/content/drive/MyDrive/people"
UNKNOWN_FACES_DIR = "/content/drive/MyDrive/unpeople"
TOLERANCE = 0.5

FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = "cnn" # hog

known_faces = []
known_names = []

# 등록하기
for name in os.listdir(KNOWN_FACES_DIR):
  for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
    image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
    encoding = face_recognition.face_encodings(image)[0]
    known_faces.append(encoding)
    known_names.append(name)
    print(known_names)
    print(known_faces)


# Step2. Face Recognition

In [ ]:
# 테스트 사진 불러오기 - 해당 얼굴위치에 대한 좌표 인코딩
print("processing unknown faces")
for filename in os.listdir(UNKNOWN_FACES_DIR):
  print(filename)
  image = face_recognition.load_image_file(f"{UNKNOWN_FACES_DIR}/{filename}")
  locations = face_recognition.face_locations(image, model= MODEL)
  encodings = face_recognition.face_encodings(image, locations)
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

  for face_encoding, face_location in zip(encodings, locations):
    results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)
    match = None
    if True in results:
      match = known_names[results.index(True)]
      print(f"Match found: {match}")

      top_left = (face_location[3], face_location[0])
      bottom_right = (face_location[1], face_location[2])

      color = [0, 255, 0]
      cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS)

      top_left = (face_location[3], face_location[2])
      bottom_right = (face_location[1], face_location[2]+22)
      cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)
      cv2.putText(image, match, (face_location[3]+10, face_location[2]+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), FONT_THICKNESS)
    cv2_imshow(image)
    cv2.waitKey(100000)
    #cv2.destroyWindow(filename)